For assignment 2, I selected the fourth Jester dataset in [http://eigentaste.berkeley.edu/dataset/] This dataset consists of two parts - a user rating dataset where 7000+ users rated distinct 158 jokes, and a jokes dataset consisting of a single column of jokes that were rated.  

The general strategy of my recommender system will be as follows:

1.  Develop a baseline for comparison.  This will be done using the same method in Project 1, where the mean of all user ratings among all jokes is used to establish a baseline.  The user ratings dataset will be split into a testing set and a training set, with a 80% - 20% split.
2.  Incorporate Content-Based filtering by using tf-idf to determine similarity between jokes.  A recommendation can be developed using a user's own joke ratings and the cosine-similarity matrix calculated with the tf-idf.
4.  Incorporate Item-Item collaborative filtering 

In [96]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

user_ratings = pd.read_csv("https://raw.githubusercontent.com/koonkimb/Data612/refs/heads/main/Project_2/data/final.csv", header = None)
user_ratings_df = user_ratings.drop(user_ratings.columns[0],axis = 1)
user_ratings_df = user_ratings_df.transpose()
user_ratings_df = user_ratings_df.reset_index()
user_ratings_df = user_ratings_df.drop('index',axis = 1)
user_ratings_df = user_ratings_df.transpose()
user_ratings_df = user_ratings_df.reset_index()
print(user_ratings_df)

user_ratings_long = user_ratings_df.melt(id_vars='index', var_name='joke', value_name='rating')
print(user_ratings_long)

      index     0     1     2     3     4     5     6     7     8  ...    148  \
0         0  99.0  99.0  99.0  99.0  99.0  99.0  99.0  99.0  99.0  ...  99.00   
1         1  99.0  99.0  99.0  99.0  99.0  99.0  99.0  99.0  99.0  ...  99.00   
2         2  99.0  99.0  99.0  99.0  99.0  99.0  99.0  99.0  99.0  ...  99.00   
3         3  99.0  99.0  99.0  99.0  99.0  99.0  99.0  99.0  99.0  ...  99.00   
4         4  99.0  99.0  99.0  99.0  99.0  99.0  99.0  99.0  99.0  ...  99.00   
...     ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...    ...   
7694   7694  99.0  99.0  99.0  99.0  99.0  99.0  99.0  99.0  99.0  ...   0.00   
7695   7695  99.0  99.0  99.0  99.0  99.0  99.0  99.0  99.0  99.0  ...  99.00   
7696   7696  99.0  99.0  99.0  99.0  99.0  99.0  99.0  99.0  99.0  ...  99.00   
7697   7697  99.0  99.0  99.0  99.0  99.0  99.0  99.0  99.0  99.0  ...   2.03   
7698   7698  99.0  99.0  99.0  99.0  99.0  99.0  99.0  99.0  99.0  ...  99.00   

        149    150    151  

In [98]:

user_ratings_dropped['rating'] = user_ratings_long['rating'].replace(99,np.nan)
user_ratings_dropped = user_ratings_dropped.dropna()
df_random = user_ratings_dropped.sample(frac = 1, random_state = 63)
split_size = int(0.8*len(df_random))
train = df_random[:split_size]
test = df_random[split_size:]

print(train)
print(test)

         index joke  rating
270614    1149   35    2.93
597375    4552   77   -2.72
264573    2807   34    0.00
146048    7466   18    7.00
494479    1743   64    0.00
...        ...  ...     ...
142383    3801   18   -3.05
1110350   1694  144    3.05
142552    3970   18   -3.74
266408    4642   34    0.00
1174114   3866  152   -5.26

[85191 rows x 3 columns]
        index joke  rating
424789   1344   55    5.12
548349   1720   71    6.95
548375   1746   71   -8.03
600614     92   78    0.00
377185   7633   48   20.00
...       ...  ...     ...
133417   2534   17  -10.00
366276   4423   47    0.00
434127   2983   56    0.00
165136   3457   21    9.63
837185   5693  108    2.31

[21298 rows x 3 columns]


Pivot back the database from long to wide format:

In [99]:
pivot_train = train.pivot(index='index', columns='joke', values='rating')
print(pivot_train)
pivot_test = test.pivot(index='index', columns='joke', values='rating')
print(pivot_test)

joke   6    7    12   14   15   16   17    18    20   21   ...   148   149  \
index                                                      ...               
0      NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN   NaN  NaN  ...   NaN   NaN   
2      NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN   NaN  NaN  ...   NaN   NaN   
3      NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN -4.59  NaN  ...   NaN   NaN   
4      NaN  NaN  NaN  NaN  NaN  NaN  NaN -7.72   NaN  NaN  ...   NaN   NaN   
5      NaN  NaN  NaN  NaN  NaN  NaN  NaN  4.39   NaN  NaN  ...   NaN  3.19   
...    ...  ...  ...  ...  ...  ...  ...   ...   ...  ...  ...   ...   ...   
7694   NaN  NaN  NaN  NaN  NaN  NaN  NaN  0.00  0.00  NaN  ...   NaN   NaN   
7695   NaN  NaN  NaN  NaN  NaN  NaN  NaN -9.51   NaN  NaN  ...   NaN   NaN   
7696   NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN   NaN  NaN  ...   NaN  8.63   
7697   NaN  NaN  NaN  NaN  NaN  NaN  NaN  2.52  5.37  NaN  ...  2.03   NaN   
7698   NaN  NaN  NaN  NaN  NaN  NaN  NaN  4.59   NaN  NaN  ...  

leaving off right here - need to fill in matrix with missing users in each set 

In [107]:
reindex = list(range(7699))
column_reindex = list(range(158))
pivot_train_reindex = pivot_train.reindex(reindex).fillna(np.nan)
pivot_train_reindex_transpose = pivot_train_reindex.transpose()
pivot_train_reindex_transpose = pivot_train_reindex_transpose.reindex(column_reindex).fillna(np.nan)
pivot_train_reindex = pivot_train_reindex_transpose.transpose()

pivot_test_reindex = pivot_test.reindex(reindex).fillna(np.nan)
pivot_test_reindex_transpose = pivot_test_reindex.transpose()
pivot_test_reindex_transpose = pivot_test_reindex_transpose.reindex(column_reindex).fillna(np.nan)
pivot_test_reindex = pivot_test_reindex_transpose.transpose()

print(pivot_train_reindex)
print(pivot_test_reindex)

joke   0    1    2    3    4    5    6    7    8    9    ...   148   149  \
index                                                    ...               
0      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...   NaN   NaN   
1      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...   NaN   NaN   
2      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...   NaN   NaN   
3      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...   NaN   NaN   
4      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...   NaN   NaN   
...    ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   ...   ...   
7694   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...   NaN   NaN   
7695   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...   NaN   NaN   
7696   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...   NaN  8.63   
7697   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  2.03   NaN   
7698   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...   NaN   NaN   

joke    150

In [108]:
train_means = np.nanmean(pivot_train_reindex.values)
print(train_means)

0.5318867016468877


In [110]:
is_na = pivot_train_reindex.isna().astype(int) # create an is_na matrix 

# create matrix of ones, then multiply by the mean, then multiply by the is_na matrix so that NaN values are replaced by 0
train_means_df = np.ones((7699,158)) * train_means * (1-is_na) 

#convert to dataframe, then replace 0 with NaN
train_means_df = pd.DataFrame(train_means_df)
train_means_df = train_means_df.replace(0, np.nan)

rmse_raw_train = (((pivot_train_reindex - train_means_df) ** 2).to_numpy())
rmse_raw_train = np.sqrt(np.nanmean(rmse_raw_train))
print(rmse_raw_train)

5.055509305757885


In [112]:
is_na_test = pivot_test_reindex.isna().astype(int) # create an is_na matrix 

# create matrix of ones, then multiply by the mean, then multiply by the is_na matrix so that NaN values are replaced by 0
test_means_df = np.ones((7699,158)) * train_means * (1-is_na_test) 

#convert to dataframe, then replace 0 with NaN
test_means_df = pd.DataFrame(test_means_df)
test_means_df = test_means_df.replace(0, np.nan)

rmse_raw_test = (((pivot_test_reindex - test_means_df) ** 2).to_numpy())
rmse_raw_test = np.sqrt(np.nanmean(rmse_raw_test))
print(rmse_raw_test)

5.09985299924901


In [113]:
user_bias = pivot_train_reindex.mean(axis=1) - train_means
print(user_bias)

index
0        3.168113
1             NaN
2        0.051447
3        0.595411
4       -0.860887
          ...    
7694    -0.101432
7695   -10.041887
7696     2.223113
7697     1.395172
7698     3.693113
Length: 7699, dtype: float64


In [176]:
joke_bias = pivot_train_reindex.mean() - train_means
print(joke_bias)

joke
0           NaN
1           NaN
2           NaN
3           NaN
4           NaN
         ...   
153   -1.211308
154   -2.060754
155   -1.884826
156   -0.555840
157   -0.545272
Length: 158, dtype: float64


In [115]:
train_means_user_bias = train_means_df.add(user_bias, axis = 0)
train_means_bias = train_means_df.add(joke_bias, axis = 1)
train_means_bias[train_means_bias > 10] = 10
train_means_bias[train_means_bias < -10] = -10
print(train_means_bias)

joke   0    1    2    3    4    5    6    7    8    9    ...       148  \
index                                                    ...             
0      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...       NaN   
1      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...       NaN   
2      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...       NaN   
3      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...       NaN   
4      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...       NaN   
...    ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...       ...   
7694   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...       NaN   
7695   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...       NaN   
7696   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...       NaN   
7697   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  0.711764   
7698   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...       NaN   

joke        149      150       151   

In [116]:
test_means_user_bias = test_means_df.add(user_bias, axis = 0)
test_means_bias = test_means_df.add(joke_bias, axis = 1)
test_means_bias[test_means_bias > 10] = 10
test_means_bias[test_means_bias < -10] = -10
print(test_means_bias)

joke   0    1    2    3    4    5    6    7    8    9    ...       148  149  \
index                                                    ...                  
0      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...       NaN  NaN   
1      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...       NaN  NaN   
2      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...       NaN  NaN   
3      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...       NaN  NaN   
4      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...       NaN  NaN   
...    ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...       ...  ...   
7694   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  0.711764  NaN   
7695   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...       NaN  NaN   
7696   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...       NaN  NaN   
7697   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...       NaN  NaN   
7698   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  

Using the user biases and joke biases, we can determine the final baseline RMSE for the training and test set.

In [118]:
rmse_bias_train = (((pivot_train_reindex - train_means_bias) ** 2).to_numpy())
rmse_bias_train = np.sqrt(np.nanmean(rmse_bias_train))
print(rmse_bias_train)

4.92605004367484


In [119]:
rmse_bias_test = (((pivot_test_reindex - test_means_bias) ** 2).to_numpy())
rmse_bias_test = np.sqrt(np.nanmean(rmse_bias_test))
print(rmse_bias_test)

4.969971634915424


Per step 2 of the plan outlined above, I can incorporate content based filtering with tf-idf.  I used scikit-learn's TfidfVectorizer to do this.

In [137]:
# Import the data
jokes_df = pd.read_csv("https://raw.githubusercontent.com/koonkimb/Data612/refs/heads/main/Project_2/data/Dataset4JokeSet.csv", header = None)
print(jokes_df)

                                                     0
0    A man visits the doctor. The doctor says "I ha...
1    This couple had an excellent relationship goin...
2    Q. What's 200 feet long and has 4 teeth?   A. ...
3    Q. What's the difference between a man and a t...
4    Q.\tWhat's O. J. Simpson's Internet address?  ...
..                                                 ...
153  Poodle: "My life is a mess. My owner is mean, ...
154  Did you hear that NASA has launched several co...
155  A bear walks into a bar and says,"I'd like a b...
156  A dog goes into a bar and orders a martini. Th...
157  I'm reading a great book about antigravity -- ...

[158 rows x 1 columns]


In [121]:
# use regex to remove any integers, as the TfidfVectorizer will convert all strings to lowercase with a function that cannot take in integers
jokes_df_string = jokes_df
jokes_df_string[0] = jokes_df_string[0].replace(r'\d+', '', regex = True)
print(jokes_df_string)

                                                     0
0    A man visits the doctor. The doctor says "I ha...
1    This couple had an excellent relationship goin...
2    Q. What's  feet long and has  teeth?   A. The ...
3    Q. What's the difference between a man and a t...
4    Q.\tWhat's O. J. Simpson's Internet address?  ...
..                                                 ...
153  Poodle: "My life is a mess. My owner is mean, ...
154  Did you hear that NASA has launched several co...
155  A bear walks into a bar and says,"I'd like a b...
156  A dog goes into a bar and orders a martini. Th...
157  I'm reading a great book about antigravity -- ...

[158 rows x 1 columns]


In [140]:
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(jokes_df_string[0])
print(vectorizer.get_feature_names_out()) # extracted features (words) from the 158 jokes
print(vectorizer.vocabulary_) # full mapping of words to index
print(X.shape) # shape of tfidf matrix
print(X) # for each joke (indexed by the first number in coordinate), shows the word (indexed by the second number in the coordinate; 
# can cross reference with vocabulary_ from above) and the tf-idf score of the word

['able' 'abortion' 'accident' ... 'younger' 'zipper' 'zo']
{'man': 1084, 'visits': 1901, 'doctor': 512, 'says': 1538, 'bad': 133, 'news': 1197, 'cancer': 264, 'alzheimer': 41, 'disease': 501, 'replies': 1464, 'thank': 1796, 'god': 758, 'don': 520, 'couple': 400, 'excellent': 603, 'relationship': 1450, 'going': 761, 'day': 440, 'came': 259, 'home': 858, 'work': 1976, 'girlfriend': 749, 'packing': 1258, 'asked': 93, 'leaving': 1001, 'told': 1823, 'heard': 826, 'awful': 124, 'things': 1801, 'possibly': 1352, 'said': 1521, 'make': 1078, 'pedophile': 1294, 'replied': 1463, 'awfully': 125, 'big': 167, 'word': 1974, 'year': 1996, 'old': 1230, 'feet': 649, 'long': 1048, 'teeth': 1781, 'row': 1508, 'willie': 1954, 'nelson': 1191, 'concert': 368, 'difference': 486, 'toilet': 1820, 'doesn': 515, 'follow': 691, 'use': 1875, 'simpson': 1615, 'internet': 911, 'address': 11, 'slash': 1632, 'backslash': 131, 'escape': 587, 'hillary': 845, 'trip': 1848, 'arkansas': 80, 'gas': 731, 'pulls': 1391, 'servi

In [141]:
cosine_sim = cosine_similarity(X,X)
print(cosine_sim.shape)
print(cosine_sim)
cosine_sim_jokes = pd.DataFrame(cosine_sim)

(158, 158)
[[1.         0.         0.         ... 0.03849341 0.04877435 0.        ]
 [0.         1.         0.         ... 0.04504245 0.         0.        ]
 [0.         0.         1.         ... 0.         0.         0.        ]
 ...
 [0.03849341 0.04504245 0.         ... 1.         0.16401938 0.        ]
 [0.04877435 0.         0.         ... 0.16401938 1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]


In [142]:
train_zero = pivot_train_reindex.replace(np.nan,0)
recommendation_train = train_zero.dot(cosine_sim_jokes)
print(recommendation_train)

            0         1         2         3    4         5         6    \
index                                                                    
0      0.000000  0.038697  0.000000  0.000000  0.0  0.000000  0.000000   
1      0.000000  0.000000  0.000000  0.000000  0.0  0.000000  0.000000   
2     -0.112501  0.316217 -0.171377  0.201677  0.0  0.004553  0.000000   
3      1.064374  0.886653  0.000000 -0.449893  0.0  1.325902  0.832003   
4      0.000000  0.046331  0.000000  0.000000  0.0  0.000000  0.000000   
...         ...       ...       ...       ...  ...       ...       ...   
7694   0.323531  0.000000  0.363074  0.000000  0.0  0.034140  0.000000   
7695   0.000000  0.000000  0.000000  0.000000  0.0  0.000000  0.000000   
7696   2.726277  2.615214  0.018710  1.541704  0.0  0.983397  0.616287   
7697   1.804813  2.475106  0.022978  1.176841  0.0  1.486106  0.746487   
7698   0.000000  0.040370  0.000000  0.000000  0.0  0.000000  0.000000   

            7         8         9    

In [143]:
test_zero = pivot_test_reindex.replace(np.nan,0)
recommendation_test = test_zero.dot(cosine_sim_jokes)
print(recommendation_test)

            0         1         2         3    4         5         6    \
index                                                                    
0      0.000000  0.000000  0.000000  0.000000  0.0  0.000000  0.000000   
1      0.000000  0.085865  0.000000  0.000000  0.0  0.000000  0.000000   
2      0.000000  0.000000  0.000000  0.000000  0.0  0.000000  0.000000   
3      0.833909 -0.023999  0.056406  0.317960  0.0  0.121353 -0.103138   
4      0.000000  0.000000  0.000000  0.000000  0.0  0.000000  0.000000   
...         ...       ...       ...       ...  ...       ...       ...   
7694   0.000000  0.000000  0.000000  0.000000  0.0  0.000000  0.000000   
7695   0.000000  0.000000  0.000000  0.000000  0.0  0.000000  0.000000   
7696   0.490537  0.567192  0.000000  0.517598  0.0  0.100755  0.000000   
7697   0.989153  0.374167  0.101836  0.639184  0.0  0.399469  0.111932   
7698   0.000000  0.000000  0.000000  0.000000  0.0  0.000000  0.000000   

            7         8         9    

In [144]:
full_train_recommendation = train_means_bias + recommendation_train
full_train_recommendation[full_train_recommendation > 10] = 10
full_train_recommendation[full_train_recommendation < -10] = -10

In [145]:
full_test_recommendation = test_means_bias + recommendation_test
full_test_recommendation[full_test_recommendation > 10] = 10
full_test_recommendation[full_test_recommendation < -10] = -10

In [146]:
rmse_bias_train = (((pivot_train_reindex - full_train_recommendation) ** 2).to_numpy())
rmse_bias_train = np.sqrt(np.nanmean(rmse_bias_train))
print(pivot_train_reindex)
print(full_train_recommendation)
print(rmse_bias_train)

joke   0    1    2    3    4    5    6    7    8    9    ...   148   149  \
index                                                    ...               
0      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...   NaN   NaN   
1      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...   NaN   NaN   
2      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...   NaN   NaN   
3      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...   NaN   NaN   
4      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...   NaN   NaN   
...    ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   ...   ...   
7694   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...   NaN   NaN   
7695   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...   NaN   NaN   
7696   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...   NaN  8.63   
7697   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  2.03   NaN   
7698   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...   NaN   NaN   

joke    150

In [147]:
rmse_bias_test = (((pivot_test_reindex - full_test_recommendation) ** 2).to_numpy())
rmse_bias_test = np.sqrt(np.nanmean(rmse_bias_test))
print(pivot_test_reindex)
print(full_test_recommendation)
print(rmse_bias_test)

joke   0    1    2    3    4    5    6    7    8    9    ...  148  149   150  \
index                                                    ...                   
0      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN   NaN   
1      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN   NaN   
2      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN   NaN   
3      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN   NaN   
4      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN   NaN   
...    ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   ...   
7694   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  0.0  NaN   NaN   
7695   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN   NaN   
7696   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN   NaN   
7697   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  2.05   
7698   NaN  NaN  NaN  NaN  NaN  NaN  NaN

Per step 3 from above, I will also incorporate item-item collaborating filtering in this recommender.  This will be done by clustering jokes using cosine similarity and finding the n nearest neighbors.  

In [155]:
cosine_sim_pivot = cosine_sim_jokes
cosine_sim_pivot = cosine_sim_pivot.reset_index()
cosine_sim_pivot = cosine_sim_pivot.melt(id_vars='index', var_name='joke', value_name='cosine_similarity_score')
print(cosine_sim_pivot)

       index joke  cosine_similarity_score
0          0    0                 1.000000
1          1    0                 0.000000
2          2    0                 0.000000
3          3    0                 0.047858
4          4    0                 0.000000
...      ...  ...                      ...
24959    153  157                 0.000000
24960    154  157                 0.000000
24961    155  157                 0.000000
24962    156  157                 0.000000
24963    157  157                 1.000000

[24964 rows x 3 columns]


In [172]:
top_5_scores = cosine_sim_pivot.set_index('index').groupby('joke')['cosine_similarity_score'].nlargest(6)
top_5_scores = top_5_scores.to_frame()
print(top_5_scores.columns.tolist()) 
top_5_scores = top_5_scores.drop(top_5_scores[top_5_scores['cosine_similarity_score'] >= 1].index)
print(top_5_scores)

['cosine_similarity_score']
            cosine_similarity_score
joke index                         
0    86                    0.430934
     102                   0.347192
     133                   0.193347
     67                    0.157579
     129                   0.157152
...                             ...
157  56                    0.250732
     20                    0.096836
     135                   0.064944
     42                    0.061482
     70                    0.060895

[805 rows x 1 columns]


In [173]:
top_5_scores = top_5_scores.reset_index()
#print(top_5_scores)
top_5_scores_matrix = top_5_scores.pivot(index = 'joke' , columns = 'index', values = 'cosine_similarity_score')
print(top_5_scores_matrix)

index  0        1    2    3    4    5    6         7    8    9    ...  148  \
joke                                                              ...        
0      NaN      NaN  NaN  NaN  NaN  NaN  NaN       NaN  NaN  NaN  ...  NaN   
1      NaN      NaN  NaN  NaN  NaN  NaN  NaN       NaN  NaN  NaN  ...  NaN   
2      NaN      NaN  NaN  NaN  NaN  NaN  NaN       NaN  NaN  NaN  ...  NaN   
3      NaN      NaN  NaN  NaN  NaN  NaN  NaN       NaN  NaN  NaN  ...  NaN   
4      0.0  0.00000  0.0  0.0  NaN  NaN  NaN       NaN  NaN  NaN  ...  NaN   
...    ...      ...  ...  ...  ...  ...  ...       ...  ...  ...  ...  ...   
153    NaN  0.09342  NaN  NaN  NaN  NaN  NaN       NaN  NaN  NaN  ...  NaN   
154    NaN      NaN  NaN  NaN  NaN  NaN  NaN  0.142005  NaN  NaN  ...  NaN   
155    NaN      NaN  NaN  NaN  NaN  NaN  NaN       NaN  NaN  NaN  ...  NaN   
156    NaN      NaN  NaN  NaN  NaN  NaN  NaN       NaN  NaN  NaN  ...  NaN   
157    NaN      NaN  NaN  NaN  NaN  NaN  NaN       NaN  NaN  NaN

In [177]:
print(joke_bias)

joke
0           NaN
1           NaN
2           NaN
3           NaN
4           NaN
         ...   
153   -1.211308
154   -2.060754
155   -1.884826
156   -0.555840
157   -0.545272
Length: 158, dtype: float64
